# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [3]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests
import os

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [5]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation = os.path.join("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df = pd.read_csv(vacation)

# Display sample data
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Ilo,PE,72.46,clear sky,-17.6394,-71.3375,Hostal Central
1,3,Cam Ranh,VN,78.76,scattered clouds,11.9214,109.1591,Khách sạn Thái Bình
2,8,Atuona,PF,77.56,light rain,-9.8000,-139.0333,Pearl Resort
3,10,Georgetown,MY,82.24,few clouds,5.4112,100.3354,Page 63 hostel
4,12,Cidreira,BR,72.75,broken clouds,-30.1811,-50.2056,Hotel Castelo


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [14]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    alpha = 0.5,
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [24]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"]=="Kununurra"]
vacation_end = vacation_df.loc[vacation_df["City"]=="Kununurra"]
vacation_stop1 = vacation_df.loc[vacation_df["City"]=="Yulara"]
vacation_stop2 = vacation_df.loc[vacation_df["City"]=="Mount Isa"]
vacation_stop3 = vacation_df.loc[vacation_df["City"]=="Cairns"]


## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [25]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end])

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
46,106,Kununurra,AU,80.58,overcast clouds,-15.7667,128.7333,Kununurra Country Club Resort
11,28,Yulara,AU,71.87,clear sky,-25.2406,130.9889,Outback Pioneer Hotel & Lodge
140,362,Mount Isa,AU,76.77,light rain,-20.7333,139.5000,Ibis Styles
249,680,Cairns,AU,77.07,overcast clouds,-16.9167,145.7667,Cairns Queens Court Accommodation
46,106,Kununurra,AU,80.58,overcast clouds,-15.7667,128.7333,Kununurra Country Club Resort


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [30]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lng","Lat"]].copy()

# Display sample data
waypoints_df

,Lng,Lat
46,128.7333,-15.7667
11,130.9889,-25.2406
140,139.5000,-20.7333
249,145.7667,-16.9167
46,128.7333,-15.7667


## Use GeoViews to create map that shows the four cities in the itinerary

In [130]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriStreet",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 2,
    alpha = 0.5,
    color = "City",
    hover_cols = ["City", "Hotel Name", "Country"]
)

In [131]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [53]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [54]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '20e0a6d57f2644eeaff3a89bb96fa6d1',
 'waypoints': '-15.7667,128.7333|-25.2406,130.9889|-20.7333,139.5|-16.9167,145.7667|-15.7667,128.7333'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [64]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing" 

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [128.7333, -15.7667], 'original_index': 0},
     {'location': [130.9889, -25.2406], 'original_index': 1},
     {'location': [139.5, -20.7333], 'original_index': 2},
     {'location': [145.7667, -16.9167], 'original_index': 3},
     {'location': [128.7333, -15.7667], 'original_index': 4}],
    'units': 'metric',
    'distance': 7896866,
    'distance_units': 'meters',
    'time': 252243.789,
    'legs': [{'distance': 2141599,
      'time': 63178.003,
      'steps': [{'from_index': 0,
        'to_index': 7,
        'distance': 156,
        'time': 11.265,
        'instruction': {'text': 'Drive southwest on Button Drive.'}},
       {'from_index': 7,
        'to_index': 33,
        'distance': 1292,
        'time': 99.602,
        'instruction': {'text': 'Turn left onto Ironwood Drive.'}},
       {'from_index': 33,
        'to_index': 49,
        'distance': 1125,
        'time': 72.651,
   

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [90]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]
legs

[[[128.733768, -15.766984],
  [128.733649, -15.767166],
  [128.733589, -15.767254],
  [128.733573, -15.767371],
  [128.733728, -15.767565],
  [128.734016, -15.767864],
  [128.734163, -15.768011],
  [128.734194, -15.768122],
  [128.734267, -15.768013],
  [128.734528, -15.767801],
  [128.735258, -15.767307],
  [128.735783, -15.766975],
  [128.736019, -15.766867],
  [128.736109, -15.766819],
  [128.736388, -15.766752],
  [128.736618, -15.766753],
  [128.736959, -15.766791],
  [128.737431, -15.766903],
  [128.738559, -15.767235],
  [128.738756, -15.767299],
  [128.738811, -15.767317],
  [128.739565, -15.767587],
  [128.740253, -15.76786],
  [128.740955, -15.768119],
  [128.741362, -15.768239],
  [128.741718, -15.768282],
  [128.742157, -15.768252],
  [128.742595, -15.768156],
  [128.74302, -15.768058],
  [128.74326, -15.768023],
  [128.743461, -15.768042],
  [128.7446, -15.768276],
  [128.745388, -15.768438],
  [128.745488, -15.768459],
  [128.745125, -15.77065],
  [128.744599, -15.774347]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [98]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for item in legs:
    for coord in item:
        longitude.append(coord[0])
        latitude.append(coord[1])
    
    

[-15.766984, -15.767166, -15.767254, -15.767371, -15.767565, -15.767864, -15.768011, -15.768122, -15.768013, -15.767801, -15.767307, -15.766975, -15.766867, -15.766819, -15.766752, -15.766753, -15.766791, -15.766903, -15.767235, -15.767299, -15.767317, -15.767587, -15.76786, -15.768119, -15.768239, -15.768282, -15.768252, -15.768156, -15.768058, -15.768023, -15.768042, -15.768276, -15.768438, -15.768459, -15.77065, -15.774347, -15.774556, -15.775458, -15.775663, -15.775814, -15.775996, -15.776147, -15.776399, -15.776712, -15.777122, -15.777364, -15.777408, -15.777957, -15.778022, -15.778357, -15.77832, -15.778356, -15.778594, -15.779168, -15.779561, -15.77964, -15.780274, -15.780554, -15.780842, -15.781273, -15.781522, -15.781613, -15.782099, -15.782658, -15.783002, -15.783188, -15.783708, -15.784637, -15.786005, -15.788055, -15.78843, -15.7888, -15.789333, -15.789439, -15.789776, -15.790259, -15.790812, -15.7912, -15.791541, -15.79186, -15.792133, -15.792475, -15.792795, -15.793258, -

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [140]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["longitude"]=longitude
route_df["latitude"]=latitude

# Display sample data
route_df.head()

,longitude,latitude
0,128.733768,-15.766984
1,128.733649,-15.767166
2,128.733589,-15.767254
3,128.733573,-15.767371
4,128.733728,-15.767565


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [147]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(rout_df)
route_path

:Path   [longitude,latitude]

In [169]:
route_path = route_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    frame_width = 700,
    frame_height = 500,
    color = "red",
)

In [170]:
route_path

:Points   [longitude,latitude]

In [172]:
# Display a composed plot by using the route_map and route_path objects
WeatherPy_travel_map = (waypoints_map * route_path)
WeatherPy_travel_map

:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Points.I  :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)
   .Points.II :Points   [longitude,latitude]